# SMULE DIAGNOSTIC 

# 1)  Deck of Cards

Suppose you to create a measurement and dashboard for how well-shuffled a deck of cards is. Suppose that shuffling will be done a very large number of times a day and, to maintain user confidence in our product, we have to make sure the results are absolutely fair.

Please keep the following considerations in mind:

<b>1) How to report the data, what logging would you use?</b>

<b>2) How would you define 'fairness' and how would you describe it in numerical form?</b>

<b>3) What dashboard(s) would you use to present this data to Product.</b>

<b>4) How would you monitor the health of this metric? What are acceptable bounds?</b>

<b>1) How to report the data, what logging would you use?</b>

My first thoughts when reading the term logging is to define what we are going to log and how.


<b>What we should log:</b>
- Logging by day.
- Log every instance of a shuffle. Meaning the final ordered state of the deck. Then order these(RANK).
- It is stated that 'shuffling will be done a very large number of times a day' so we should count how many times the deck was shuffled that day.
- If we are drawing cards I would log the ordered state of the deck the cards are being drawn from. 
- Log the order of each draw and lastly how many total cards were being drawn.

Depending on how we would want to split the data multiple tables can be used. 

<b>How we will log:</b>
- Keep the logging in a database 
    - E.g. Amazon Redshift or any other kind of RDBMS so we may access is with SQL.
- If shuffles are being done on an external app or system we would need to transfer the data to a database. 
    - E.g. Amazon S3 buckets for data transfer and storage. We could also log directly into the database.

By having the data accessible we can have it readily available for dashboards and manipulation.

In [1]:
# Dependancies
import math
import random

In [2]:
# Factorials can be used to show total number of outcomes. With a deck of 4 cards we would have 24 outcomes.
print('Total outcomes for a deck of 4:',math.factorial(4))

# Total number of outcomes for a standard 52 card deck.
print('Total outcomes for a deck of 52:',math.factorial(52))

Total outcomes for a deck of 4: 24
Total outcomes for a deck of 52: 80658175170943878571660636856403766975289505440883277824000000000000


In [3]:
# For simplicity I numbered the cards 1 to 13.
# List comprehension to create a mock unopened deck.
deck = [str(x)+' '+str(y) for x in ['spade','diamond','club','heart'] for y in range(1,14)]

# Showing the deck.
deck[0:16]

['spade 1',
 'spade 2',
 'spade 3',
 'spade 4',
 'spade 5',
 'spade 6',
 'spade 7',
 'spade 8',
 'spade 9',
 'spade 10',
 'spade 11',
 'spade 12',
 'spade 13',
 'diamond 1',
 'diamond 2',
 'diamond 3']

In [4]:
# Creates a dictionary to record all the shuffles you want to have done that day.
def shuffler(shuffles):
    dict={}
    for x, num in enumerate(range(0,shuffles)):
        dict[num] = [random.choice(deck) for x in range (0,53)]
    return dict

In [5]:
# 2 Shuffles
Jan_01_2018 = shuffler(2)

In [6]:
# These records could be kept in one table
Jan_01_2018

{0: ['spade 3',
  'club 5',
  'spade 11',
  'club 11',
  'club 7',
  'diamond 9',
  'diamond 8',
  'heart 3',
  'diamond 5',
  'spade 7',
  'spade 10',
  'heart 2',
  'heart 6',
  'spade 13',
  'spade 5',
  'heart 3',
  'heart 6',
  'heart 6',
  'heart 10',
  'club 2',
  'heart 12',
  'spade 7',
  'club 7',
  'spade 11',
  'club 10',
  'heart 10',
  'heart 11',
  'spade 7',
  'diamond 11',
  'spade 4',
  'spade 6',
  'diamond 6',
  'heart 5',
  'spade 12',
  'club 13',
  'heart 9',
  'diamond 3',
  'diamond 11',
  'spade 12',
  'heart 8',
  'spade 1',
  'club 11',
  'heart 12',
  'club 4',
  'heart 4',
  'heart 12',
  'spade 1',
  'spade 13',
  'spade 10',
  'club 9',
  'club 8',
  'diamond 5',
  'club 10'],
 1: ['heart 11',
  'diamond 10',
  'heart 12',
  'heart 12',
  'club 4',
  'club 11',
  'spade 4',
  'club 7',
  'heart 10',
  'heart 12',
  'spade 6',
  'spade 3',
  'diamond 2',
  'spade 2',
  'diamond 13',
  'club 5',
  'spade 10',
  'diamond 11',
  'heart 4',
  'diamond 11',
  

In [7]:
# Drawing could be kept in another table.
# For any given dictionary we can book keep the order of draws and how many draws
def drawing(date_dict, draws):
    dict={}
    for x, num in enumerate (range(0,len(date_dict))):
        dict[num] = date_dict[x][0:draws]
    return dict

In [8]:
# Each entry is a deck order that has been shuffled followed by the draw order.
drawing(Jan_01_2018,5)

{0: ['spade 3', 'club 5', 'spade 11', 'club 11', 'club 7'],
 1: ['heart 11', 'diamond 10', 'heart 12', 'heart 12', 'club 4']}

<b>2) How would you define 'fairness' and how would you describe it in numerical form?</b>

Fairness is an open ended term, one that is tough to define and needs context.
By assigning terms to how I would use fainess, I could make a loose definition.

<b>Ways to achieve 'fairness':</b>
- To maintain user confidence in our product I believe choosing a <b>minimum number of shuffles</b> we can at a minimum have a baseline for ourselves and our consumers. With the references listed below <b>7</b> seems to be where deck shuffles start to become truly random

- We will remove any entry that has a <b>duplicate</b> on that day.

<b>Numerical 'fairness':</b>
- To define a quick metric for fairness we can calculate the difference for each card being at a minimum X postions (depending on how 'random' we want to be we can increase X) away from its starting point. <b>We can take an average of the absolute value of the differences</b> to give us a numerical definition of fairness for this example. The higher the score the more 'fair' it will be.

In [9]:
# I could consolidate these functions into one, for easiness to read I left them seperate.
example = ['a','b','c','d','e','f']

In [10]:
# Function to rank the entries in the example list.
def ranking_list(lst):
    dict={}
    for x, num in enumerate(lst):
        dict[num]=x
    return dict

In [11]:
# Ranking the entries in the example list.
a = ranking_list(example)

In [12]:
# Shuffle the list
random.shuffle(example)
b = ranking_list(example)

In [13]:
# Show the original list. (Unshuffled deck)
a

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5}

In [14]:
# Show the shuffled deck and rank.
b

{'d': 0, 'f': 1, 'b': 2, 'e': 3, 'a': 4, 'c': 5}

In [15]:
# Creates dictionary of differences.
def fairness(a,b):
    return dict((x, b.get(x, 0) - a.get(x, 0)) for x in set(a)|set(b))

In [16]:
# Show the differences.
c = fairness(a,b)
c

{'e': -1, 'd': -3, 'a': 4, 'c': 3, 'b': 1, 'f': -4}

In [17]:
# Find the absolute value sum to find the 'fairness' metric.
print('Fairness score:',sum([abs(value) for value in c.values()]))

Fairness score: 14


# Resources
https://www.dartmouth.edu/~chance/teaching_aids/Mann.pdf

http://rspa.royalsocietypublishing.org/content/456/2002/2561

<b>3) What dashboard(s) would you use to present this data to Product.</b>

Delivering a dashboard to Product I would keep the dashboard as simple as possible.
I'll be using Tableau terminology, meaning each table or plot is a worksheet and the final product is a dashboard.
All three worksheets would be combined into one dashboard and then delivered to Product.

The overall dashboard would have a date, number of shuffles and number of draws filters.

<b>Table:</b>

I would build this table to have rows by date. 

For columns:
- I would have the number of times shuffled that day or discrete number of times shuffled.
- Followed by average fairness that day.
- Average cards drawn or discrete number of cards drawn.

<b>Line Graph</b>
- To display consistency I would plot fairness vs. date. Date being X and average fairness by day being Y.

<b>Scatter Plot per Day:</b>
- This worksheet would be more granular. I would display each shuffle as a point on the scatter plot to see where it fell on the 'fairness' scale.

<b>4) How would you monitor the health of this metric? What are acceptable bounds?</b>

To monitor health I would have a certain benchmark for daily average of 'fairness' as well for weeks and months. If at any point we were not hitting the benchmarks we know we are not shuffling enough, correctly or something within our application has gone wrong.

As for acceptable bounds.
With the 'fairness' ranking I've chosen I don't belive there is a need for an upper bounch only a lower. Each lower bound can be determined and used for the health benchmarks mentioned above. By having the dashboard on a daily basis we can monitor progress and see if any issues have arisen that day. 

# 2) Hairs on Head
<b>1) What is the probability that at least two people in the USA have the same number of hairs on their heads?</b>

<b>2) What is the probability that someone else in the USA has the same number of hairs on their head as you?</b>

<b>3) What percent of people in the USA has someone in the USA that has the same number of hairs on their head as them?</b>

<b>Definitions:</b> 

I researched and used the <b>pigeonhole principle</b> and specifically the <b>birthday problem</b> to prove this.

The pigoen hole principle states:
- That if n items are put into m containers, with n > m, then at least one container must contain more than one item.

The birthday problem states:
- For a set of n randomly chosen people, what is the probability that some pair of them will have the same birthday?

<b>Variables:</b> 

For this example I will be using the average number of hairs a typical human is born with, ~100,000 so I'll go with <b>100,000</b>.

There are 326,766,748 people in United States in 2018. To simplify things I'll round up to <b>327,000,000</b>.

<b>1) What is the probability that at least two people in the USA have the same number of hairs on their heads?</b>

<b>Answer:</b> The probability will be 100%

We can use our knowledge about the pigeon hole principal and the birthday problem to solve the hair problem. 

Based on the pigeon hole principle, any set of people containing more than 100,000 people will have at least two people who have the same number of hairs. Because 327,000,000 > 100,000. With this in mind we know that there will be a 100% chance that at least two people will have this match.

So let us put in our variables to the bithday problem question.
For a set of 327,000,000 people what is the probability that at least two of them will have the same number of hairs (100,000).

In [18]:
# Dependancies
import math

# Using the birthday problem formual x! / ((x-n)! * x^n).
# Created a simple function taking two parameters, hairs_x for amount of hairds or containers and people_n for the set
def hair_probability(hairs_x,people_n):
    try:
        p = 1 - math.factorial(hairs_x)/(math.factorial(hairs_x-people_n)*(hairs_x**people_n))
        return p*100
    except:
        print('A larger samples size than containers results in a negative.')

In [19]:
# Showing math for the birthday problem. At 23 people in a room there is a 50% chance of having the same birthday.
hair_probability(365,23)

50.72972343239854

In [20]:
# Mathmatecially prove probability is 100%
hair_probability(100000,100000)

100.0

In [21]:
# Trying a populdation of 326,000,000. Larger samples sizes than containers results in a negative.
hair_probability(100000,327000000)

A larger samples size than containers results in a negative.


<b>2) What is the probability that someone else in the USA has the same number of hairs on their head as you?</b>

<b>Answer:</b> This is the same question, two people in the United States having the same amount of hairs.
This is proven above that the probability will be 100%

<b>3) What percent of people in the USA has someone in the USA that has the same number of hairs on their head as them?</b>

<b>Answer:</b> Again, the same question worded differently. As long as we are finding two people and n > m the probability will remain at 100%. So if everyone in the USA has a matching counterpart, the answer to this question will also be 100%.

# 3) SQL

With the three tables provided below, please provide queries that answer these  questions. If you think there is a significantly more efficient method than the one you chose, please describe it and comment on your choice:

<b>1) How many users installed?</b>

<b>2) What proportion came through Paid channels?</b>

<b>3) What was the average revenue per user generated between the day a user installed (Day 1) and seven days later (Day 8 Cumulative Revenue per User)? E.g.</b>
  - 1000 installs
  - 100 subscriptions
  - $500 revenue
  - $0.50 RPU

<b>4) Calculate Day 2 retention based on the number of times a user sang on Day 1? E.g.</b> 
  - 50 users didn’t sing and had 30% Day 2 retention, 100 users sang once and had 35% Day 2 retention

<b>5) What is the average time gap between 'Singing' events on the same calendar date? Does this interval change as users sing more songs? E.g.</b>
  - User 1 sang for the first time at 12:10 and sang for the second time at 12:30, so their time gap between Song 1  and Song 2 is 20 minutes.
  - User 1 sang again at 12:40, so their second gap 10 minutes, their average gap is 15 minutes, and the gap shrank with after singing another song.

<b>Users</b>
- user_id
- install_date
- channel ('Paid', 'Organic')

<b>Revenue: One record for each subscription</b>
- user_id
- calendar_date
- revenue (in U.S. dollars e.g. $9.99)

<b>Events: One record for each instance of these user actions (each time a user sings, each time user is active)</b>
- user_id
- calendar_date
- event_name ('Singing', 'Active User')
- timestamp


My Answers are written with alias tables with the 'with' statement from PostgreSQL/AWS syntax.

<b>1) How many users installed?</b>

In [ ]:
# I assume all users that are in the two other tables should be in the 'Users' tables
# Could include WHERE install_date is NOT NULL or other where statements like this to prevent null values.

'''
SELECT 
 COUNT(distinct user_id) 
FROM Users
;
    
'''

<b>2) What proportion came through Paid channels?</b>

In [ ]:
# Could also do count of a case statement where channel is paid divided by count of all.

'''
SELECT 
 COUNT(channel)/(SELECT COUNT(channel) from Users)
FROM Users
WHERE channel = 'paid'
;

'''

<b>3) What was the average revenue per user generated between the day a user installed (Day 1) and seven days later (Day 8 Cumulative Revenue per User)?</b>

In [ ]:
# 7 day revenue calculation.
'''
SELECT 
 SUM(r.revenue)/COUNT(DISTINCT u.user_id)
FROM Users u
LEFT JOIN Revenue r ON u.user_id = r.user_id
WHERE r.calendar_date BETWEEN u.install_date AND DATEADD('day', 7, u.install_date)
GROUP BY u.user_id
;


'''

<b>4) Calculate Day 2 retention based on the number of times a user sang on Day 1?</b> 

In [ ]:
# Calculating retention based on example grouping by amount of times sung by user. 

'''
with counts as (
SELECT
user_id,
install_date
COUNT(event_name) as times_sung
FROM Users u
LEFT JOIN Events e
ON u.user_id = e.user_id
AND e.timestamp < DATEADD('day', 1, install_time)
AND event_name = 'Singing'
GROUP BY 1, 2)

SELECT
times_sung,
COUNT(DISTINCT c.user_id) as active_first_day,
COUNT(DISTINCT e.user_id) as retention,
COUNT(DISTINCT e.user_id) / COUNT(DISTINCT c.user_id) as retention_pct
FROM counts c
LEFT JOIN Events e
ON e.user_id = c.user_id
AND e.timestamp > DATEADD('day', 1, install_time)
AND e.timestamp < DATEADD('day', 2', install_time)
;

'''

<b>5) What is the average time gap between 'Singing' events on the same calendar date? Does this interval change as users sing more songs?</b>

In [ ]:
# Lag/Lead to calculate the differences.

# What is the average time gap between “Singing” events on the same calendar date?
'''
SELECT
 AVG(LEAD(timestamp) OVER (PARTITION BY user_id, calendar_date ORDER BY timestamp) - timestamp)
WHERE event_name = 'Singing'
;
'''

# Does this interval change as users sing more songs?
'''
with interval_diff as (
SELECT
 calendar_date
 , user_id
 , LEAD(timestamp) OVER (PARTITION BY user_id, calendar_date ORDER BY timestamp) - timestamp as time_gap
 , RANK() OVER (PARTITION BY user_id, calendar_date ORDER BY timestamp) as ranked
WHERE event_name = 'Singing'
GROUP BY 1,2
;)

SELECT
 ranked,
 avg(time_gap)
FROM interval_diff
GROUP BY 1
;

'''